In [1]:
import os
import urllib3
import json
import re
import numpy as np
import pdal
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point, mapping

In [2]:
MINX, MINY, MAXX, MAXY = [-93.759055, 41.925015, -93.766155, 41.935015]
polygon = Polygon(((MINX, MINY), (MINX, MAXY), (MAXX, MAXY), (MAXX, MINY), (MINX, MINY)))

In [40]:
class UsgsLidar:
    
    def __init__(self, path = "https://s3-us-west-2.amazonaws.com/usgs-lidar-public/", pipeline_json_path: str="../pipeline.json") -> None:
            
        self.path = path
        self.input_epsg = 3857
        self.txt = self.read_txt("../data/filenames.txt")
        self.a = self.read_json("../pipeline.json")
    
    def read_json(self, json_path):
        try:
            with open(json_path) as js:
                json_obj = json.load(js)
            return json_obj

        except FileNotFoundError:
            print('File not found.')
        
    def fetch_polygon_boundaries(self, polygon: Polygon):
        polygon_df = gpd.GeoDataFrame([polygon], columns=['geometry'])

        polygon_df.set_crs(epsg=4326, inplace=True)
        polygon_df['geometry'] = polygon_df['geometry'].to_crs(epsg=3857)
        minx, miny, maxx, maxy = polygon_df['geometry'][0].bounds

        polygon_input = 'POLYGON(('
        xcords, ycords = polygon_df['geometry'][0].exterior.coords.xy
        for x, y in zip(list(xcords), list(ycords)):
            polygon_input += f'{x} {y}, '
        polygon_input = polygon_input[:-2]
        polygon_input += '))'

        return f"({[minx, maxx]},{[miny,maxy]})", polygon_input
    
    def read_csv(self, csv_path, missing_values=["n/a", "na", "undefined"]):
        try:
            df = pd.read_csv(csv_path, na_values=missing_values)
            return df

        except FileNotFoundError:
            print('File not found.')
            
    def fetch_pipeline (self, region: str, polygon: Polygon):
        url = f"{self.path}{region}/ept.json"
        boundary, poly = self.fetch_polygon_boundaries(polygon)
        
        self.a['pipeline'][0]['filename']= f"{self.path}{region}/ept.json"
        self.a['pipeline'][0]['polygon'] = poly
        self.a['pipeline'][0]['bounds'] = boundary
        pipeline = pdal.Pipeline(json.dumps(self.a))
        
        return pipeline
    
    def execute_pipeline(self, polygon: Polygon, epsg=4326, region: str = "IA_FullState"):
        
        pipeline = self.fetch_pipeline(region, polygon)

        try:
            pipeline.execute()
            print(f'Pipeline executed successfully.')
            return pipeline
        except RuntimeError as e:
            print('Pipeline execution failed')
            print(e)
    
    
    def create_gpd_df(self, epsg, pipe):
        try:
            cloud = []
            elevations =[]
            geometry=[]
            for row in pipe.arrays[0]:
                lst = row.tolist()[-3:]
                cloud.append(lst)
                elevations.append(lst[2])
                point = Point(lst[0], lst[1])
                geometry.append(point)
            gpd_df = gpd.GeoDataFrame(columns=["elevation", "geometry"])
            gpd_df['elevation'] = elevations
            gpd_df['geometry'] = geometry
            gpd_df = gpd_df.set_geometry("geometry")
            gpd_df.set_crs(epsg = epsg, inplace=True)
            return gpd_df
        except RuntimeError as e:
            print(e)

    def fetch_region_data(self, polygon: Polygon, epsg=4326):
        pipeline = self.execute_pipeline(polygon, epsg)
        return self.create_gpd_df(epsg, pipeline)
    
    def read_txt(self, txt_path) -> list:
        try:
            with open(txt_path, "r") as f:
                text_file = f.read().splitlines()
            
            return text_file

        except Exception as e:
            print(e)
            
    def fetch_name_and_year(self, location: str) -> tuple:
        
        location = location.replace('/', '')
        regex = '20[0-9][0-9]+'
        match = re.search(regex, location)
        if(match):
          return (location[:match.start() - 1], location[match.start():match.end()])
        else:
          return (location, None)
    
   
    def fetch_metadata(self):
    
        metadata = pd.DataFrame(columns=['filename', 'region',
                          'year', 'xmin', 'xmax', 'ymin', 'ymax', 'points'])

        index = 0
        for lists in self.txt:
          r = urllib3.PoolManager().request('GET', self.path + f + "ept.json")
          if r.status == 200:
            j = json.loads(r.data)
            region, year = get_name_and_year(lists)

            metadata = metadata.append({
                'filename': lists.replace('/', ''),
                'region': region,
                'year': year,
                'xmin': j['bounds'][0],
                'xmax': j['bounds'][3],
                'ymin': j['bounds'][1],
                'ymax': j['bounds'][4],
                'points': j['points']}, ignore_index=True)

            metadata.to_csv("../data/metadata.csv")
        
        return(metadata)

In [41]:
US = UsgsLidar()
df = US.read_csv("../data/iowa.csv")
shape, poly = US.fetch_polygon_boundaries(polygon)
print(poly)

POLYGON((-10437210.259858532 5149753.664381643, -10437210.259858532 5151249.971344454, -10438000.628243161 5151249.971344454, -10438000.628243161 5149753.664381643, -10437210.259858532 5149753.664381643))


In [5]:
files = US.read_txt("../data/filenames.txt")
len(files)

1598

In [39]:
US.fetch_region_data(polygon)

Pipeline executed successfully.


,elevation,geometry
0,311.06,POINT (-93.76450 41.93500)
1,310.98,POINT (-93.76451 41.93500)
2,311.06,POINT (-93.76454 41.93501)
3,310.99,POINT (-93.76455 41.93501)
4,311.09,POINT (-93.76453 41.93499)
...,...,...
913969,309.80,POINT (-93.76211 41.93347)
913970,317.36,POINT (-93.76370 41.93348)
913971,308.19,POINT (-93.76052 41.93466)
913972,309.04,POINT (-93.76211 41.93465)


In [28]:
metadata

,filename,region,year,xmin,xmax,ymin,ymax,points
0,AK_BrooksCamp_2012,AK_BrooksCamp,2012,-17347360,-17321558,8065364,8091166,529285317
1,AK_Coastal_2009,AK_Coastal,2009,-15730544,-15691854,10937407,10976097,55711772
2,AK_Fairbanks-NSBorough_2010,AK_Fairbanks-NSBorough,2010,-16471700,-16381190,9519129,9609639,1266097458
3,AK_Juneau_2012,AK_Juneau,2012,-15014449,-14943073,8012267,8083643,2211557952
4,AK_Kenai_2008,AK_Kenai,2008,-16906356,-16570284,8303726,8639798,14054081685
5,AK_MatanuskaSusitna-Lot1_2011,AK_MatanuskaSusitna-Lot1,2011,-16782341,-16678201,8663224,8767364,4988109681
6,AK_MatanuskaSusitna-Lot2_2011,AK_MatanuskaSusitna-Lot2,2011,-16768328,-16390088,8706126,9084366,41053208083
7,AK_NomeCreek_2010,AK_NomeCreek,2010,-16391082,-16348578,9681153,9723657,94592518
8,AK_NorthSlope_B10_2018,AK_NorthSlope_B10,2018,-18152228,-18139030,10978280,10991478,148956845
9,AK_NorthSlope_B11_2018,AK_NorthSlope_B11,2018,-18571233,-18557621,10544583,10558195,122307792
